# **Data Preparation**

In [51]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GrainImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, mode='train'):

        self.dataframe = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.mode = mode

        self.class_to_index = {'wheat': 0, 'oats': 1, 'flax': 2, 'barley': 3}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        if self.mode == 'train':
            class_name = self.dataframe.iloc[idx, 1]
            label = self.class_to_index[class_name]
            label = torch.tensor(label, dtype=torch.long)
        else:
            # Dummy label (-1), поскольку test.csv не содержит классов
            label = torch.tensor(-1, dtype=torch.long)
        return image, label

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Model Architecture**

In [52]:
# Model definition
class GrainClassifier(nn.Module):
    def __init__(self):
        super(GrainClassifier, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Input channels = 3 (RGB)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(256 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 4)  # 4 classes: wheat, oats, flax, barley

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = x.view(-1, 256 * 8 * 8)
        x = self.dropout(x)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = self.fc2(x)
        return x

index_to_class = {0: 'wheat', 1: 'oats', 2: 'flax', 3: 'barley'}

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_csv_path = '/content/drive/MyDrive/NeuralNetworks/train/train.csv'
test_csv_path = '/content/drive/MyDrive/NeuralNetworks/test/test.csv'
train_img_dir = '/content/drive/MyDrive/NeuralNetworks/train/images'
test_img_dir = '/content/drive/MyDrive/NeuralNetworks/test/images'

train_dataset = GrainImageDataset(csv_file=train_csv_path, img_dir=train_img_dir, transform=transform, mode='train')
test_dataset = GrainImageDataset(csv_file=test_csv_path, img_dir=test_img_dir, transform=transform, mode='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = 4
model = GrainClassifier().to(device)

# **Optimizer**

In [53]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

# **Loss function**

In [54]:
loss_func = nn.CrossEntropyLoss()

# **Training**

In [55]:
# Training loop
def train(model, train_loader, loss_func, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Execute training
train(model, train_loader, loss_func, optimizer, num_epochs=10, device=device)

Epoch 1/10, Loss: 1.1995505736424372
Epoch 2/10, Loss: 1.0483199672384576
Epoch 3/10, Loss: 0.9327336890356881
Epoch 4/10, Loss: 0.8583368868618221
Epoch 5/10, Loss: 0.7726396983796424
Epoch 6/10, Loss: 0.7647321892308665
Epoch 7/10, Loss: 0.7444684338438642
Epoch 8/10, Loss: 0.6840629541611933
Epoch 9/10, Loss: 0.617866656937442
Epoch 10/10, Loss: 0.5915774896249666


# **Evaluation**

In [56]:
def evaluate(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().tolist())
    return predictions

# Execute evaluation
predictions = evaluate(model, test_loader, device=device)

# Convert numeric predictions to class names
class_predictions = [index_to_class[pred] for pred in predictions]

# Load the test dataframe, add class predictions, and save to a new CSV
test_df = pd.read_csv(test_csv_path)
test_df['class'] = class_predictions
test_df.to_csv('/content/drive/MyDrive/NeuralNetworks/predictions.csv', index=False)

